Statistical Analysis of Results
===============================
This will be an examnination of results. Just statistical analysis nothing else.

# Table of Contents
1. [Preliminaries](#1.-preliminaries)
2. [Kling-Gupta Efficiency](#2.-kling-gupta-efficiency)
   * [isoNet](#isoNet-KGE)
   * [isoP](#isoP-KGE)
3. [Root Mean Square Error](#3.-root-mean-square-error)
   * [isoNet](#isoNet-RMSE)
   * [isoP](#isoP-RMSE)
4. [Combining Results](#4.-combining-results)

# 1. Preliminaries
This is the setup for the rest of the analysis.

In [1]:
# Library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Load in the isotope data that it was trained on, and extract the lat lon of the individual stations
isotope_data = pd.read_csv('Isoscape_Data.csv')
isotope_data.drop_duplicates(subset=['Station'], inplace=True)
isotope_data.reset_index()

station_coord = isotope_data[['Station', 'Lat', 'Long']]
station_coord.set_index('Station', inplace=True)
station_coord = station_coord.to_dict(orient='index')
station_coord

{'OTT': {'Lat': 45.32, 'Long': -75.67},
 'RES': {'Lat': 74.43, 'Long': -94.59},
 'HAL': {'Lat': 68.47, 'Long': -81.15},
 'ALR': {'Lat': 82.31, 'Long': -62.17},
 'EUR': {'Lat': 79.59, 'Long': -85.56},
 'CAM': {'Lat': 69.6, 'Long': -105.8},
 'BAB': {'Lat': 47.98, 'Long': -55.82},
 'SNA': {'Lat': 63.52, 'Long': -116.0},
 'SKT': {'Lat': 52.1, 'Long': -106.43},
 'ELA': {'Lat': 49.67, 'Long': -93.72},
 'SAT': {'Lat': 48.78, 'Long': -123.13},
 'HAB': {'Lat': 46.29, 'Long': -64.15},
 'CPA': {'Lat': 49.82, 'Long': -74.97},
 'BON': {'Lat': 49.38, 'Long': -82.12},
 'EGB': {'Lat': 44.23, 'Long': -79.77},
 'GOB': {'Lat': 53.32, 'Long': -60.42},
 'EST': {'Lat': 51.67, 'Long': -110.2}}

In [3]:
# Load in results into a pandas dataframe
results = pd.read_csv('results_test.csv')

# Convert the day of year column and year column into a datetime object
results['date'] = pd.to_datetime(results['Year'].astype(str) + '-' + results['Day'].astype(str), format='%Y-%j')

results = results.drop(columns=['Year', 'Day'])

# Create new column for the station name and fill it with the station name based off the lat lon and the station_coord dataframe
for stat in station_coord:
    results.loc[results['Lat'] == station_coord[stat]['Lat'], 'Station'] = stat

# Change the date to start on the first of each month, instead of the second
results.date = results.date - pd.Timedelta('1D')

results.head()

,Lat,Long,Alt,Precipitation (kg/m^2/s),Temperature (K),Predictions,Actual,date,Station
0,82.31,-62.17,30,0.000000,240.15315,-32.946890,-34.07,2003-11-01,ALR
1,63.52,-116.00,241,0.000000,267.63196,-26.461382,-23.35,2003-11-01,SNA
2,46.29,-64.15,45,0.000056,274.37950,-14.373904,-8.56,2003-11-01,HAB
3,79.59,-85.56,10,0.000000,240.76923,-32.530334,-33.20,2003-11-01,EUR
4,45.32,-75.67,114,0.000043,274.78870,-12.844803,-11.10,2003-11-01,OTT


# 2. Kling-Gupta Efficiency
In this I will be breaking down the Kling-Gupta Efficiency (KGE) into its components and then analyzing the results. Finally also storing them in a dataframe for later examination.

## isoNet KGE

In [4]:
# Create a new dataframe consisting of the mean and standard deviation of the isotope values for each station, and for the Prediction column and Actual column
station_stats = results["Predictions"].groupby(results["Station"]).agg(['mean', 'std'])
station_stats.rename(columns={'mean': 'Pred_Mean', 'std': 'Pred_Std'}, inplace=True)
station_stats['Actual_Mean'] = results["Actual"].groupby(results["Station"]).mean()
station_stats['Actual_Std'] = results["Actual"].groupby(results["Station"]).std()

# Create a new column consisting of the pearson correlation coefficient between the Predictions and the Actual values for each station
station_stats['Corr'] = results["Predictions"].groupby(results["Station"]).corr(results["Actual"])
station_stats

,Pred_Mean,Pred_Std,Actual_Mean,Actual_Std,Corr
Station,,,,,
ALR,-30.308303,4.333187,-29.480897,6.405524,0.662516
BAB,-11.007288,3.049190,-9.901625,3.180380,0.690499
BON,-15.642331,4.698027,-15.687750,6.229460,0.845811
CAM,-24.175668,4.581605,-24.989722,6.387800,0.889819
CPA,-16.688394,4.984807,-15.816750,5.421716,0.842194
ELA,-16.596430,4.406398,-15.259167,5.625312,0.839306
EUR,-29.071648,4.848347,-29.439054,6.699601,0.654679
GOB,-16.792867,4.003135,-15.539875,3.850453,0.855974
HAB,-12.255528,3.146063,-11.117500,4.066346,0.570372


In [5]:
# Create new dataframe just for the KGE components and values
kge = pd.DataFrame(columns=['alpha', 'beta', 'r', 'kge'], index=station_stats.index)

# Fill in the KGE dataframe with the alpha value (variablility ratio)
kge['alpha'] = station_stats['Pred_Std'] / station_stats['Actual_Std']

# Fill in the KGE dataframe with the beta value (bias ratio)
kge['beta'] = (station_stats['Pred_Mean'] - station_stats['Actual_Mean']) / station_stats['Actual_Mean']

# Fill in the KGE dataframe with the r value (correlation coefficient)
kge['r'] = station_stats['Corr']

# Fill in the KGE dataframe with the KGE value
kge['kge'] = 1 - np.sqrt((kge['alpha'] - 1)**2 + (kge['beta'])**2 + (1 - kge['r'])**2)

kge

,alpha,beta,r,kge
Station,,,,
ALR,0.676477,0.028066,0.662516,0.531651
BAB,0.958750,0.111665,0.690499,0.668396
BON,0.754163,-0.002895,0.845811,0.709796
CAM,0.717243,-0.032576,0.889819,0.694791
CPA,0.919415,0.055109,0.842194,0.814437
ELA,0.783316,0.087637,0.839306,0.716355
EUR,0.723677,-0.012480,0.654679,0.557556
GOB,1.039653,0.080631,0.855974,0.830244
HAB,0.773683,0.102364,0.570372,0.503736


## isoP KGE
Performing the same calculations but this time just for the isoP results

In [6]:
# Load in Data from isoP
isoP = pd.read_csv('isoP_Output.csv')

# Change longitudes to be positive
isoP['Lon'] = isoP['Lon'] * -1

# Add in the station name to the isoP dataframe with the station_coord dataframe
for stat in station_coord:
    isoP.loc[isoP['Lat'] == station_coord[stat]['Lat'], 'Station'] = stat

# Combine Month and Year into a single column and convert to datetime object
isoP['date'] = pd.to_datetime(isoP['Year'].astype(str) + '-' + isoP['Month'].astype(str), format='%Y-%m')
isoP.drop(columns=['Year', 'Month'], inplace=True)

# Reorder the columns
isoP = isoP[['date', 'Station', 'Lat', 'Lon', 'isoP']]

# Remove any rows in isoP that do not have a corresponding row in the results dataframe
isoP = isoP[isoP['date'].isin(results['date'])]

# Merge the isoP dataframe with the results dataframe on the date and station columns, keeping only the actual and isoP columns
isoP = isoP.merge(results, on=['date', 'Station'], how='inner')

isoP.drop(columns=['Lat_y', 'Long', 'Precipitation (kg/m^2/s)', 'Temperature (K)', 'Predictions'], inplace=True)
isoP.rename(columns={'Lat_x': 'Lat'}, inplace=True)
isoP

,date,Station,Lat,Lon,isoP,Alt,Actual
0,2003-11-01,ALR,82.31,62.17,-33.750136,30,-34.070
1,2003-12-01,ALR,82.31,62.17,-34.854257,30,-36.590
2,2004-01-01,ALR,82.31,62.17,-36.576392,30,-42.085
3,2004-02-01,ALR,82.31,62.17,-34.860847,30,-38.230
4,2004-03-01,ALR,82.31,62.17,-38.579669,30,-43.600
...,...,...,...,...,...,...,...
472,2006-11-01,CPA,49.82,74.97,-15.369955,382,-13.420
473,2006-12-01,CPA,49.82,74.97,-17.134303,382,-19.550
474,2007-01-01,CPA,49.82,74.97,-20.370909,382,-20.710
475,2007-02-01,CPA,49.82,74.97,-25.603669,382,-30.170


In [7]:
# Now we can perform the same analysis as we did with the isoNet data
isoP_stats = isoP["isoP"].groupby(isoP["Station"]).agg(['mean', 'std'])
isoP_stats.rename(columns={'mean': 'isoP_Mean', 'std': 'isoP_Std'}, inplace=True)
isoP_stats['Actual_Mean'] = isoP["Actual"].groupby(isoP["Station"]).mean()
isoP_stats['Actual_Std'] = isoP["Actual"].groupby(isoP["Station"]).std()
isoP_stats['Corr'] = isoP["isoP"].groupby(isoP["Station"]).corr(isoP["Actual"])

isoP_stats

,isoP_Mean,isoP_Std,Actual_Mean,Actual_Std,Corr
Station,,,,,
ALR,-29.440770,6.081501,-29.480897,6.405524,0.714146
BAB,-9.537126,2.476090,-9.901625,3.180380,0.582041
BON,-15.630624,5.112508,-15.687750,6.229460,0.836987
CAM,-24.846960,6.013302,-24.989722,6.387800,0.876755
CPA,-15.703708,4.391651,-15.816750,5.421716,0.913618
ELA,-16.352054,5.178594,-15.259167,5.625312,0.809264
EUR,-29.072991,6.522508,-29.439054,6.699601,0.767087
GOB,-15.493962,3.651505,-15.539875,3.850453,0.852105
HAB,-10.323760,2.899864,-11.117500,4.066346,0.682479


In [8]:
# Now the KGE values
isoP_kge = pd.DataFrame(columns=['alpha', 'beta', 'r', 'kge'], index=isoP_stats.index)

# Fill in the KGE dataframe with the alpha value (variablility ratio)
isoP_kge['alpha'] = isoP_stats['isoP_Std'] / isoP_stats['Actual_Std']

# Fill in the KGE dataframe with the beta value (bias ratio)
isoP_kge['beta'] = (isoP_stats['isoP_Mean'] - isoP_stats['Actual_Mean']) / isoP_stats['Actual_Mean']

# Fill in the KGE dataframe with the r value (correlation coefficient)
isoP_kge['r'] = isoP_stats['Corr']

# Fill in the KGE dataframe with the KGE value
isoP_kge['kge'] = 1 - np.sqrt((isoP_kge['alpha'] - 1)**2 + (isoP_kge['beta'])**2 + (1 - isoP_kge['r'])**2)

isoP_kge

,alpha,beta,r,kge
Station,,,,
ALR,0.949415,-0.001361,0.714146,0.709701
BAB,0.778552,-0.036812,0.582041,0.525569
BON,0.820698,-0.003641,0.836987,0.757646
CAM,0.941373,-0.005713,0.876755,0.863402
CPA,0.810011,-0.007147,0.913618,0.791173
ELA,0.920588,0.071622,0.809264,0.781331
EUR,0.973567,-0.012435,0.767087,0.765262
GOB,0.948331,-0.002955,0.852105,0.843312
HAB,0.713137,-0.071396,0.682479,0.566171


In [9]:
# Combine the isoNet and isoP KGE dataframes into a single dataframe
kge = pd.merge(isoP_kge, kge, left_index=True, right_index=True, suffixes=('_isoP', '_isoNet'))
kge = kge[['kge_isoP', 'kge_isoNet']]
kge

,kge_isoP,kge_isoNet
Station,,
ALR,0.709701,0.531651
BAB,0.525569,0.668396
BON,0.757646,0.709796
CAM,0.863402,0.694791
CPA,0.791173,0.814437
ELA,0.781331,0.716355
EUR,0.765262,0.557556
GOB,0.843312,0.830244
HAB,0.566171,0.503736


# 3. Root Mean Square Error
In this I will be calculating the Root Mean Square Error (RMSE) 

## isoNet RMSE
Just for the current isoNet results

In [10]:
# In station_stats, calculate the RMSE for each station
station_stats['RMSE'] = np.sqrt(((results['Predictions'] - results['Actual']) ** 2).groupby(results['Station']).mean())
station_stats

,Pred_Mean,Pred_Std,Actual_Mean,Actual_Std,Corr,RMSE
Station,,,,,,
ALR,-30.308303,4.333187,-29.480897,6.405524,0.662516,4.808673
BAB,-11.007288,3.049190,-9.901625,3.180380,0.690499,2.663086
BON,-15.642331,4.698027,-15.687750,6.229460,0.845811,3.329882
CAM,-24.175668,4.581605,-24.989722,6.387800,0.889819,3.178751
CPA,-16.688394,4.984807,-15.816750,5.421716,0.842194,3.043422
ELA,-16.596430,4.406398,-15.259167,5.625312,0.839306,3.318936
EUR,-29.071648,4.848347,-29.439054,6.699601,0.654679,5.029578
GOB,-16.792867,4.003135,-15.539875,3.850453,0.855974,2.433460
HAB,-12.255528,3.146063,-11.117500,4.066346,0.570372,3.585050


## isoP RMSE
Now for the isoP results

In [11]:
isoP_stats['RMSE'] = np.sqrt(((isoP['isoP'] - isoP['Actual']) ** 2).groupby(isoP['Station']).mean())
isoP_stats

,isoP_Mean,isoP_Std,Actual_Mean,Actual_Std,Corr,RMSE
Station,,,,,,
ALR,-29.440770,6.081501,-29.480897,6.405524,0.714146,4.669464
BAB,-9.537126,2.476090,-9.901625,3.180380,0.582041,2.652300
BON,-15.630624,5.112508,-15.687750,6.229460,0.836987,3.367999
CAM,-24.846960,6.013302,-24.989722,6.387800,0.876755,3.059712
CPA,-15.703708,4.391651,-15.816750,5.421716,0.913618,2.248999
ELA,-16.352054,5.178594,-15.259167,5.625312,0.809264,3.498192
EUR,-29.072991,6.522508,-29.439054,6.699601,0.767087,4.468797
GOB,-15.493962,3.651505,-15.539875,3.850453,0.852105,2.023738
HAB,-10.323760,2.899864,-11.117500,4.066346,0.682479,3.044396


# 4. Combining Results

In [12]:
# Create a dataframe for the RMSE values and the KGE values
final_stats = pd.DataFrame(columns=['RMSE_isoNet', 'RMSE_isoP', 'KGE_isoNet', 'KGE_isoP'], index=station_stats.index)

# Fill in the final_stats dataframe with the RMSE and KGE values
final_stats['RMSE_isoNet'] = station_stats['RMSE']
final_stats['RMSE_isoP'] = isoP_stats['RMSE']
final_stats['KGE_isoNet'] = kge['kge_isoNet']
final_stats['KGE_isoP'] = kge['kge_isoP']

final_stats

,RMSE_isoNet,RMSE_isoP,KGE_isoNet,KGE_isoP
Station,,,,
ALR,4.808673,4.669464,0.531651,0.709701
BAB,2.663086,2.652300,0.668396,0.525569
BON,3.329882,3.367999,0.709796,0.757646
CAM,3.178751,3.059712,0.694791,0.863402
CPA,3.043422,2.248999,0.814437,0.791173
ELA,3.318936,3.498192,0.716355,0.781331
EUR,5.029578,4.468797,0.557556,0.765262
GOB,2.433460,2.023738,0.830244,0.843312
HAB,3.585050,3.044396,0.503736,0.566171


In [13]:
# Export the final_stats dataframe to a csv file
final_stats.to_csv('results_stats.csv')